Ascon-128

Duplex with double keyed initialization and finalization.

320-bit inner permutation.

128-bit key, nonce and tag, and 64-bit data block.

6 inner rounds, 12 otherwise.

Ascon-p

SPN with a round constant addition, substitution layer, and linear layer. 

In [2]:
def asconp(state, a, b, is_a):
    if is_a:
        for i in a:
            r = i
            constant_addition(state, r)
            substitution_layer(state)
            linear_layer(state)
    else: 
        for i in b:
            r = i + a - b
            constant_addition(state, r)
            substitution_layer(state)
            linear_layer(state)


f0 = 240

e1 = 225

d2 = 210

c3 = 195

b4 = 180

a5 = 165

96 = 150

87 = 135

78 = 120

69 = 105

5a = 90

4b = 75

In [3]:
def constant_addition(state, r):
    
    constant = 240 - (15 * r)

    state[2] = xor_64int(state[2], constant)
    

Ascon 5-bit S-box

| $x$ | 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | a | b | c | d | e | f | 10 | 11 | 12 | 13 | 14 | 15 | 16 | 17 | 18 | 19 | 1a | 1b | 1c | 1d | 1e | 1f |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| $S(x)$ | 4 | b | 1f | 14 | 1a | 15 | 9 | 2 | 1b | 5 | 8 | 12 | 1d | 3 | 6 | 1c | 1e | 13 | 7 | e | 0 | d | 11 | 18 | 10 | c | 1 | 19 | 16 | a | f | 17 |

$x_{0}$ MSB

In [5]:
def substitution_layer(state):
    bitstate = []

    for i in 5:
        bitstate.append(word_to_bits(state[i]))

    for i in 64:
        #x4 lsb
        input = bitstate[i][4] + (bitstate[i][3] * 2) + (bitstate[i][2] * 4) + (bitstate[i][1] * 8) + (bitstate[i][0] * 16)

        output = sbox(input)

        bitstate[i][4] = output % 2
        output = output // 2

        bitstate[i][3] = output % 2
        output = output // 2

        bitstate[i][2] = output % 2
        output = output // 2

        bitstate[i][1] = output % 2
        output = output // 2

        bitstate[i][0] = output % 2
        output = output // 2

    for i in 5:
        state[i] = bits_to_int(bitstate[i])
        

$x_{i} ← ∑_i(x_i)$

$x_{0} ← ∑_0(x_0) = x_0 ⊕ (x_0 >>> 19) \oplus (x_0 >>> 28)$

$... = x_1 ⊕ (x_1 >>> 61) \oplus (x_1 >>> 39)$

$... = x_2 ⊕ (x_2 >>> 1) \oplus (x_2 >>> 6)$

$... = x_3 ⊕ (x_3 >>> 10) \oplus (x_3 >>> 17)$

$... = x_4 ⊕ (x_4 >>> 7) \oplus (x_4 >>> 41)$


In [6]:
def linear_layer(state):

    tmp = xor_64int(state[0], right_circular_shift(state[0], 19))
    tmp = xor_64int(tmp, right_circular_shift(state[0], 28))
    state[0] = tmp

    tmp = xor_64int(state[1], right_circular_shift(state[1], 61))
    tmp = xor_64int(tmp, right_circular_shift(state[1], 39))
    state[1] = tmp

    tmp = xor_64int(state[2], right_circular_shift(state[2], 1))
    tmp = xor_64int(tmp, right_circular_shift(state[2], 6))
    state[2] = tmp

    tmp = xor_64int(state[3], right_circular_shift(state[3], 10))
    tmp = xor_64int(tmp, right_circular_shift(state[3], 17))
    state[3] = tmp

    tmp = xor_64int(state[4], right_circular_shift(state[4], 7))
    tmp = xor_64int(tmp, right_circular_shift(state[4], 41))
    state[4] = tmp